In [ ]:
# 基于iwencai flag的超短线策略: 根据flag期望及分布回测某段时间的策略表现

from opentrader.core.ticker import *
from opentrader.common.db import db_ot
from opentrader.common.utils import d2dt
from opentrader.agents.ths.api import symbol_convert3
from pymongo.errors import *
from datetime import datetime, timedelta

basistag = "2013_2014" # 基于哪段数据的统计进行计算
targettag = "2015" # 目标年份
# 回测的时间段
start_date = gen_time('2015-01-01 00:00:00').date()
end_date = gen_time('2015-12-17 00:00:00').date()
date = start_date
dates = []
while date <= end_date:
    if TradeCalendar.check_date(date):
        dates.append(date)
    date += timedelta(1)

In [ ]:
# 将k线数据缓存在内存中
kday_cache = {} # {'...symbol...':[{}, {}, {}]}
cur_year = 0
for date in dates:
    if date.year != cur_year:
        print(date.year)
        cur_year = date.year
    else:
        print('.', end='')
    result = db_ot.ths_kday.find({'date':d2dt(date)})
    for each in result:
        if each['symbol'] not in kday_cache:
            kday_cache[each['symbol']] = []
        kday_cache[each['symbol']].append(each)

In [ ]:
# 从k线缓存获取数据的工具函数
# only the first day has a calculated "percent" value
def get_k_day_from_date(symbol, date, count=10):
    if symbol not in kday_cache:
        return None
    for (i, each) in enumerate(kday_cache[symbol]):
        if each['date'] == d2dt(date):
            rtn = kday_cache[symbol][i:(i+count)]
            if i > 0:
                if kday_cache[symbol][i-1]['close'] == 0:
                    rtn[0]['percent'] = 0.0
                else:
                    rtn[0]['percent'] = (float(rtn[0]['close']) / float(kday_cache[symbol][i-1]['close']) - 1) * 100
            else:
                rtn[0]['percent'] = 0.0
            return rtn
    return None
    #result = db_ot.ths_kday.find({'symbol':symbol, 'date':{'$gte':d2dt(date)}}).sort('date')
    #return list(result)

#print(get_k_day_from_date("SH601139", datetime(2015, 1, 5, 0, 0)))

In [ ]:
# 读取flag期望数据
import json
f = open('iwencai_flag_performance_oneday_'+basistag+'.json', 'r')
oneday_result = json.loads(f.read())
f.close()
f = open('iwencai_flag_performance_onedayf_'+basistag+'.json', 'r')
onedayf_result = json.loads(f.read())
f.close()
print(len(oneday_result))
print(len(onedayf_result))

In [ ]:
# 计算历史大盘指数并与上证指数比较
f = open('iwencai_flags_withtype.json', 'r')
flags_withtype = json.loads(f.read())
f.close()
indexs = []   # 指数1: 根据buy与sale类型flag次数累计后求平均 
indexs2 = []  # 指数2: 根据所有flag每次出现求和平均后的大盘期望
szzs = []
last_sh = 3000.0
for date in dates: 
    index = 0
    index2 = 0
    totalflags = 0
    for key in ("buy", "sale", "event", "zxst"):
        for flag in flags_withtype[key]:
            result = db_ot.iwencai_flag_history.find_one({'flag':flag, 'date':d2dt(date)})
            if not result:
                continue
            if key == 'buy':
                index += len(result['symbols'])
            elif key == 'sale':
                index -= len(result['symbols'])
            if flag in oneday_result:
                index2 += oneday_result[flag]['avr'] * len(result['symbols'])
            totalflags += len(result['symbols'])
            
    index = index/2666
    if totalflags:
        index2 = index2/totalflags
    else:
        index2 = 0.0
    indexs.append(index)
    indexs2.append(index2)
    sh = float(db_ot.ths_kday.find_one({'symbol':'SH000001', 'date':d2dt(date)})['close'])
    szzs.append(sh)
    print("%s, index:%f, index2:%f, szzs:%f" % (date, index, index2, (sh/last_sh-1.0)*100))
    last_sh = sh
    # 2015重要股灾日： 5.5, 5.28, 6.16, 6.26, 7.27, 8.18, 9.15, 10.21, 11.27

In [ ]:
# 计算每天期望值前五名的个股，并记录第二天的平均涨幅
import json, operator
f = open('iwencai_flags.json', 'r')
flags = json.loads(f.read())
f.close()

index2_max = max(indexs2)
date_performance = []
date_performance2 = [] # 考虑仓位变化的持仓股票performance
indexs3 = [] # 每天可入手的5支股票期望的平均值
for (idate, date) in enumerate(dates):
    position_counts = round(indexs2[idate]/index2_max/0.2)
    symbol_means = {}
    symbol_max_flag = {}
    for flag in flags:
        if flag in ('停牌','涨停'):# 此处过滤 “涨停” 是为了滤掉ST股票的影响
            continue
        #if '光头' in flag:
        #    continue
        result = db_ot.iwencai_flag_history.find_one({'flag':flag, 'date':d2dt(date)})
        if not result:
            continue
        for symbol in result['symbols']:
            if symbol not in symbol_means:
                symbol_means[symbol] = []
            try:
                symbol_means[symbol].append(onedayf_result[flag]['avr'])
                if symbol not in symbol_max_flag or onedayf_result[flag]['avr'] > onedayf_result[symbol_max_flag[symbol]]['avr']:
                    symbol_max_flag[symbol] = flag
            except KeyError:
                continue
    for symbol,means in symbol_means.items():
        if len(means):
            symbol_means[symbol] = sum(means)/len(means)
        else:
            symbol_means[symbol] = 0.0
    symbol_means_sorted = sorted(symbol_means.items(), key=operator.itemgetter(1), reverse=True)
    performance = []
    stocks = []
    i = 0
    for (symbol, mean) in symbol_means_sorted:
        sym = symbol_convert3(symbol)
        #if sym.startswith("SZ300"):
        #    continue
        try:
            kdays = get_k_day_from_date(sym, date)
        except ZeroDivisionError:
            continue
        if kdays is None or len(kdays) < 2 or float(kdays[0]['percent']) > 9.8:
            continue
        if float(kdays[0]['close']) == 0.0:
            continue
        if idate > 0 and float(kdays[0]['percent']) == 0.0: #第一天发行的股票
            continue
        if kdays[0]['date'] != d2dt(date): # 停牌
            continue
        else:
            performance.append((float(kdays[1]['close']) / float(kdays[0]['close']) - 1)*100)
            stocks.append((sym, symbol_max_flag[symbol], mean))
        i += 1
        if i >= 5:
            break
    # 2015重要股灾日： 5.5, 5.28, 6.16, 6.26, 7.27, 8.18, 9.15, 10.21, 11.27
    if len(performance) == 0:
        date_performance.append(0.0)
        date_performance2.append(0.0)
        indexs3.append(0.0)
        print("%s: zero result" % (date))
    else:
        date_performance.append(sum(performance) / len(performance))
        indexs3.append(sum([each[2] for each in stocks]) / len(stocks))
        position_counts = min(position_counts, len(performance))
        if position_counts == 0:
            date_performance2.append(0.0)
        else:
            date_performance2.append(sum(performance[:position_counts]) / position_counts)
        print("%s: available stocks: %s, index2: %f, index3: %f, performance: %f, stocks: %s, position: %f" % (date, len(performance), indexs2[idate], indexs3[idate], sum(performance) / len(performance), stocks, round(indexs2[idate]/index2_max/0.2)*0.2))


In [ ]:
fund_basic = []
init = 1.0
for each in date_performance:
    if each < -4.0:
        each = -4.0
    if each > 10.0:
        each = 10.0
    init *= 0.998
    init = init * (1.0 + each/100.0)
    fund_basic.append(init)

fund = []
positions = []
init = 1.0
trade_days = 0
for (i, each) in enumerate(date_performance2):
    if indexs2[i] < 0.0:
        fund.append(init)
        positions.append(0.0)
        continue
    if each < -4.0:
        each = -4.0
    if each > 10.0:
        each = 10.0
    #init = init * (1.0 + each/100.0)
    init *= 0.998
    position = float(round(indexs2[i]/index2_max/0.2)*0.2)
    init = init * (1.0 - position) + init * position * (1.0 + each/100.0)
    fund.append(init)
    positions.append(position)
    trade_days += 1

# 确定仓位的方案
#f = open('iwencai_flag_trade_dates.json', 'r')
#trade_dates = json.loads(f.read())
#f.close()
trade_dates = []
recent_highs = []
recent_high_num = 20
recent_lows = []
recent_low_num = 25
status = True
for (idate, each) in enumerate(indexs2):
    if status == True:
        if len(recent_lows) == 0 or each < min(recent_lows):
            status = False
        else:
            trade_dates.append(dates[idate])
    else:
        if len(recent_highs) == 0 or each > max(recent_highs):
            status = True
            trade_dates.append(dates[idate])
    
    # update recent highs and lows
    recent_highs.append(each)
    if len(recent_highs) > recent_high_num:
        recent_highs.pop(0)
    
    recent_lows.append(each)
    if len(recent_lows) > recent_low_num:
        recent_lows.pop(0)
print(len(trade_dates))

fund_1or0 = []
init = 1.0
for (idate, each) in enumerate(date_performance):
    #if str(dates[idate]) not in trade_dates:
    if dates[idate] not in trade_dates:
        fund_1or0.append(init)
        continue
    if each < -4.0:
        each = -4.0
    if each > 10.0:
        each = 10.0
    init *= 0.998
    init = init * (1.0 + each/100.0)
    fund_1or0.append(init)
    
print(index2_max)
print(len(dates))
print(len(date_performance))

%matplotlib inline
from opentrader.core.crawler import *
from datetime import datetime
from opentrader.jupyter.lib.plot import *
from opentrader.ceres.account import Account
from opentrader.ceres.trade import *
# 2015重要股灾日： 5.5, 5.28, 6.16, 6.26, 7.27, 8.18, 9.15, 10.21, 11.27
#draw_line("index", dates, [{"value":indexs}])
draw_line("index2", dates, [{"value":indexs2}])
draw_line("index3", dates, [{"value":indexs3}])

#draw_line("position", dates, [{"value":positions}])
draw_line("%snet: -4%%stop, 2/1000fee, full position" % (targettag), dates, [{"value":fund_basic}])
draw_line("%snet: -4%%stop, 2/1000fee, dynamic position" % (targettag), dates, [{"value":fund}])
draw_line("%snet: -4%%stop, 2/1000fee, 1or0 position" % (targettag), dates, [{"value":fund_1or0}])